# Sparkify Project Workspace
This workspace contains a tiny subset (128MB) of the full dataset available (12GB). Feel free to use this workspace to build your project, or to explore a smaller subset with Spark before deploying your cluster on the cloud. Instructions for setting up your Spark cluster is included in the last lesson of the Extracurricular Spark Course content.

You can follow the steps below to guide your data analysis and model building portion of this project.

In [7]:
# import libraries
# import libraries
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf

#from pyspark.sql.functions import isnan
import datetime

import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt


ModuleNotFoundError: No module named 'pyspark'

In [3]:
# create a Spark session
# create a Spark session
spark = SparkSession \
       .builder \
       .appName("Sparkify-X") \
       .getOrCreate()

NameError: name 'SparkSession' is not defined

In [4]:
spark.sparkContext.getConf().getAll()

NameError: name 'spark' is not defined

# Load and Clean Dataset
In this workspace, the mini-dataset file is `mini_sparkify_event_data.json`. Load and clean the dataset, checking for invalid or missing data - for example, records without userids or sessionids. 

In [ ]:
path = "mini_sparkify_event_data.json"
user_log = spark.read.json(path)

In [ ]:
user_log.printSchema()

In [ ]:
col_list = user_log.schema.names

In [ ]:
user_log.show(5)

In [ ]:
for col in col_list:
 print('Empty Strings for Column:',col)   
 print(user_log.filter(user_log[col] == '').count())
 print('-----------------------------') 

In [ ]:
null_val_dict = {}
for col in col_list:
 print('Null Values for Column:',col)   
 print(user_log.filter(user_log[col].isNull()).count())
 null_val_dict[col] = user_log.filter(user_log[col].isNull()).count()
 print('-----------------------------')   

In [ ]:
null_val_dict

In [ ]:
user_log.count()

# Exploratory Data Analysis
When you're working with the full dataset, perform EDA by loading a small subset of the data and doing basic manipulations within Spark. In this workspace, you are already provided a small subset of data you can explore.

### Define Churn

Once you've done some preliminary analysis, create a column `Churn` to use as the label for your model. I suggest using the `Cancellation Confirmation` events to define your churn, which happen for both paid and free users. As a bonus task, you can also look into the `Downgrade` events.

### Explore Data
Once you've defined churn, perform some exploratory data analysis to observe the behavior for users who stayed vs users who churned. You can start by exploring aggregates on these two groups of users, observing how much of a specific action they experienced per a certain time unit or number of songs played.

# Feature Engineering
Once you've familiarized yourself with the data, build out the features you find promising to train your model on. To work with the full dataset, you can follow the following steps.
- Write a script to extract the necessary features from the smaller subset of data
- Ensure that your script is scalable, using the best practices discussed in Lesson 3
- Try your script on the full data set, debugging your script if necessary

If you are working in the classroom workspace, you can just extract features based on the small subset of data contained here. Be sure to transfer over this work to the larger dataset when you work on your Spark cluster.

# Modeling
Split the full dataset into train, test, and validation sets. Test out several of the machine learning methods you learned. Evaluate the accuracy of the various models, tuning parameters as necessary. Determine your winning model based on test accuracy and report results on the validation set. Since the churned users are a fairly small subset, I suggest using F1 score as the metric to optimize.

# Final Steps
Clean up your code, adding comments and renaming variables to make the code easier to read and maintain. Refer to the Spark Project Overview page and Data Scientist Capstone Project Rubric to make sure you are including all components of the capstone project and meet all expectations. Remember, this includes thorough documentation in a README file in a Github repository, as well as a web app or blog post.